# Introduction to time series analysis and forecasting

The dataset consists of multipe time series representing an aggregated number of daily views for multiple Wikipedia articles, starting from July, 1st, 2015 up until August 2Oth, 2017.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from main.utils.utils_methods import clean
plt.rcParams['figure.figsize'] = [10, 5]


In [ ]:
# Import the 90 traffiic time series
data = pd.read_csv("../compet_data/public/train.csv", index_col = "Day", parse_dates = True)
data = data.asfreq("D")
data.head()

In [ ]:
# Choose the time series to work with: "series-1", "series-2", ..., "series-90" 
myts = "series-10"
DT = data[myts]
DT.plot()


In [ ]:
    series = DT
    series_cleaned_1, outliers_1 = clean(series)
    series_cleaned_2, outliers_2 = clean(series_cleaned_1)

    plt.subplot(3, 1, 1)
    plt.plot(series)
    plt.plot(series.loc[outliers_1], 'ro')

    plt.subplot(3, 1, 2)
    plt.plot(series_cleaned_1)
    plt.plot(series_cleaned_1.loc[outliers_2], 'ro')

    plt.subplot(3, 1, 3)
    plt.plot(series_cleaned_2)

In [ ]:
# Handle outliers
DT_cleaned_1, _ = clean(DT)
DT_cleaned_2, _ = clean(DT_cleaned_1)
DT.loc[:] = DT_cleaned_2
DT.plot()


Read the code associated to the function "clean", and briefly describe what it is doing.

In [ ]:
DT = DT.to_frame()
DT["d"] = data.index.day.to_numpy()
DT["m"] = data.index.month.to_numpy()
DT["y"] = data.index.year.to_numpy()
DT["w"] = data.index.weekday.to_numpy()
DT["wy"] = data.index.weekofyear.to_numpy()
DT.head()

In [ ]:
# Seasonal plots
## Day of the month
patterns_day_month = DT[[myts, "d", "m", "y"]].pivot_table(index=['d'], columns=['m', 'y'])
plt.plot(patterns_day_month)
plt.show()
#patterns_day_month = patterns_day_month.div(patterns_day_month.median(axis=1), axis=0)
avg_day_month = np.nanmean(patterns_day_month, axis = 1)
std_day_month = np.nanstd(patterns_day_month, axis = 1)
plt.plot(pd.DataFrame({"avg": avg_day_month, "std+": avg_day_month +  std_day_month, "std-": avg_day_month -  std_day_month}))
plt.show()

Using the previous code, produce a seasonal plot for the day of the week.

In [ ]:
# Seasonal plots
## Day of the week
patterns_day_week = DT[[myts, "wy", "y", "w"]].pivot_table(index=['w'], columns=['wy', 'y'])
plt.plot(patterns_day_week)
#patterns_day_week = patterns_day_week.div(patterns_day_week.median(axis=1), axis=0)
avg_day_week = np.nanmean(patterns_day_week, axis = 1)
std_day_week = np.nanstd(patterns_day_week, axis = 1)
pd.DataFrame({"avg": avg_day_week, "std+": avg_day_week +  std_day_week, "std-": avg_day_week -  std_day_week}).plot()
plt.show()

Produce lagged scatterplots for multiple lags. What do you observe? Add the diagonal for a better visualization.

In [ ]:
# Lag plot

def lag_plot(series, lag = -1, ls = 'r.'):
    y_lag = series.shift(lag)
    plt.plot(series, series, 'b-')
    plt.plot(y_lag, series, ls)
    plt.show()

lag_plot(DT[myts], lag = 1)
lag_plot(DT[myts], lag = 1, ls = 'r-')
lag_plot(DT[myts], lag = 3)
lag_plot(DT[myts], lag = 7)

DT[myts].shift(1)

## Autocorrelation
Plot the autocorrelation function (ACF) for the first 20 lags, and interpret the results. 
Recompute the ACF after applying a seasonal difference. 




In [ ]:
from statsmodels.tsa.stattools import acf 
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(DT[myts], lags= 20, alpha=0.05)
plt.show()
plot_acf(np.diff(DT[myts], 7), lags= 20, alpha=0.05)
plt.show()
print(acf(DT[myts]))

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
acorr_ljungbox(DT[myts], period = 7)


## White noise process

- Generate a time series with 500 observations from a white noise process with zero mean and unit standard deviation.
- Plot the tiime series
- Compute and plot the ACF for 50 lags
- Did you expect to see such results? Why?



In [ ]:

white_noise = np.random.normal(loc=0, scale=1, size=500)
plt.plot(white_noise)
plot_acf(white_noise, lags=50)
acorr_ljungbox(white_noise)

## Transformations

Compute various Box-Cox transformations. Which one do you think is more appropriate?

In [ ]:
DT[myts].transform(lambda x: x ** 0.5).plot()
plt.show()
DT[myts].transform(lambda x: x ** 0.333).plot()
plt.show()
DT[myts].transform(lambda x: np.log(x)).plot()


In [ ]:
from scipy.stats import boxcox
x, opt_lambda = boxcox(DT[myts])
print(opt_lambda)
plt.plot(x)

## Time series decomposition 


Decompose the time series into trend, seasonal and remainder components. Does it help you to understand the data?

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(DT[myts])
result.plot()
# print(result.trend)
# print(result.seasonal)
# print(result.resid)

In [ ]:
# STL decomposition
from statsmodels.tsa.seasonal import STL
stl = STL(DT[myts], period = 7, robust = True, seasonal = 7*100 + 1)
result = stl.fit()
fig = result.plot()
plt.show()

# Deaseasonlized data
deaseasonlized_data = result.trend + result.resid
deaseasonlized_data.plot()
plt.show()


#
np.log(deaseasonlized_data).plot()
plt.show()
np.log(deaseasonlized_data).diff().plot()
plt.show()

# 
deaseasonlized_data.diff(7).plot()
plt.show()



Produce forecasts for the next 21 days using simple methods. Analyze the residuals. Compare the forecast accuracy for different accuracy measures.

In [ ]:
HORIZON = 21

series = DT[myts]
series_train = series[:-21]
series_test = series[-21:]
plt.plot(series_test)


# Mean
meanf = series_train.mean()

## Training set
df_resid = (series_train- meanf)
plt.close()
plt.hist( df_resid.values )



In [ ]:
HORIZON = 21

series = DT[myts]
series_train = series[:-21]
series_test = series[-21:]
plt.plot(series_train)
plt.plot(series_test)

In [ ]:
#x = [(i, i-7) for i in range(7, len(series_train)) ]
#print(x[:14])
fit_snaive = [series_train[i-7] for i in range(7, len(series_train)) ]
fit_snaive = fit_snaive + list(series_train[:7].values)
len(fit_snaive)


In [ ]:
# Residuals (in-sample one-step ahead predictions)

## Mean forecasts
fit_mean = [series_train[:i].mean() for i in range(0, len(series_train)) ]
resid_mean = series_train.values - fit_mean

## Naive forecasts
fit_naive = [series_train[i-1] for i in range(1, len(series_train)) ]
fit_naive.insert(0, series_train[0])
resid_naive = series_train.values - fit_naive

## Seasonal naive forecasts
fit_snaive = [series_train[i-7] for i in range(7, len(series_train)) ]
fit_snaive = list(series_train[:7].values) + fit_snaive
resid_snaive = series_train.values - fit_snaive

## Plots
plt.figure()
plt.hist(resid_mean, alpha=0.2, label='mean', color='orange')
plt.figure()
plt.hist(resid_naive, alpha=0.2, label='naive', color='green')
plt.figure()
plt.hist(resid_snaive, alpha=0.2, label='snaive', color='red')
plt.legend(loc='upper right')


fit_mean = pd.DataFrame(fit_mean, index = series_train.index)
fit_naive = pd.DataFrame(fit_naive, index = series_train.index)
fit_snaive = pd.DataFrame(fit_snaive, index = series_train.index)


plt.figure()
plt.plot(series_train)
plt.plot(fit_mean, color='orange')

plt.figure()
plt.plot(series_train)
plt.plot(fit_naive, color='green')

plt.figure()
plt.plot(series_train)
plt.plot(fit_snaive, color='red')


In [ ]:
# Out-of-sample forecasts

future = series[-21:]

## Mean
f_mean = pd.DataFrame([meanf for h in range(0, HORIZON) ], index = future.index)

## Naive
f_naive = pd.DataFrame([series_train[-1] for h in range(0, HORIZON) ], index = future.index)

## Seasonal naive
f_snaive = [series_train[-HORIZON+h] for h in range(0, HORIZON) ]
f_snaive = pd.DataFrame(f_snaive, index = future.index)

plt.plot(future, label='true')
plt.plot(f_mean, label='mean')
plt.plot(f_naive, label='naive')
plt.plot(f_snaive, label='snaive')
plt.legend(loc='upper right')



In [ ]:
from main.utils.utils import *

print(smape(future.values, f_mean.values))
print(smape(future.values, f_naive.values))
print(smape(future.values, f_snaive.values))
print("----")
print(mape(future.values, f_mean.values))
print(mape(future.values, f_naive.values))
print(mape(future.values, f_snaive.values))